## Imports

In [1]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

### Scene imports

In [2]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import MeshcatVisualizerParams, Role, GeometrySet, CollisionFilterDeclaration
from pydrake.geometry.optimization import CspaceFreePolytope, SeparatingPlaneOrder
from pydrake.multibody.rational import RationalForwardKinematics
from ciris_plant_visualizer import CIrisPlantVisualizer
import numpy as np

from pydrake.all import (HPolyhedron, AngleAxis,
                         VPolytope, Sphere, Ellipsoid, InverseKinematics,
                         Hyperellipsoid, Simulator, Box, RobotDiagramBuilder)
import mcubes

import visualization_utils as viz_utils

import pydrake.symbolic as sym
from pydrake.all import  TriangleSurfaceMesh, Rgba, SurfaceTriangle, Sphere
from scipy.linalg import null_space
import time

### Widgets imports

In [3]:
from ipywidgets import widgets
from functools import partial

### MOSEK License imports

In [4]:
import os
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.common import Parallelism


## Set up the scene

In [6]:
builder = RobotDiagramBuilder(time_step=0.0)
plant = builder.plant()
scene_graph = builder.scene_graph()
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot and object models from SDFs
gripper = parser.AddModels(file_name="my_sdfs/wsg_2dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))


plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()

# Penetration allowed between gripper and cap

# Gripper inner collision
gripper_geometries = GeometrySet()
for i in range(len(plant.GetBodyIndices(gripper))):
    body_index = plant.GetBodyIndices(gripper)[i]
    body_geometries = inspector.GetGeometries(
        plant.GetBodyFrameIdOrThrow(body_index))
    gripper_geometries.Add(geometry_ids=body_geometries)
scene_graph.collision_filter_manager().Apply(
    CollisionFilterDeclaration().ExcludeWithin(gripper_geometries))

# Gripper and cap collision
cap_geometries = GeometrySet()
for i in range(len(plant.GetBodyIndices(cap))):
    body_index = plant.GetBodyIndices(cap)[i]
    body_geometries = inspector.GetGeometries(
        plant.GetBodyFrameIdOrThrow(body_index))
    cap_geometries.Add(geometry_ids=body_geometries)
scene_graph.collision_filter_manager().Apply(
    CollisionFilterDeclaration().ExcludeWithin(cap_geometries))


# Set initial configuration
# plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
#     plant_context, -0.025
# )
# # plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
# #     plant_context, 0.025
# # )

# visualizer.visualize_collision_constraint(factor = 10, N=50, iso_surface = 0.5, wireframe = True)
# # visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(np.pi/2), np.array([0,0,1])))

print(plant.num_positions())
print(plant.GetPositionNames())

# start meshcat for task space and delete the default cube
meshcat_task_space = StartMeshcat()
meshcat_task_space.Delete()

# start meshcat for tangent configuration space
meshcat_tcspace = StartMeshcat()
meshcat_tcspace.Delete()
builder_tcspace = DiagramBuilder()
plant_tcspace, scene_graph_tcspace = AddMultibodyPlantSceneGraph(
    builder_tcspace, time_step=0.0)
plant_tcspace.Finalize()


# start visualizers
visualizer_task_space = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat_task_space
)
viz_role = Role.kIllustration

visualizer_tcspace = MeshcatVisualizer.AddToBuilder(
    builder_tcspace,
    scene_graph_tcspace,
    meshcat_tcspace
)


# Build the diagrams
diagram_task_space = builder.Build()
diagram_context_task_space = diagram_task_space.CreateDefaultContext()

diagram_tcspace = builder_tcspace.Build()
diagram_context_tcspace = diagram_tcspace.CreateDefaultContext()

plant_context = plant.GetMyContextFromRoot(diagram_context_task_space)


# Publish the visualisation in the task space
diagram_task_space.ForcedPublish(diagram_context_task_space)


# Initialize the simulator
# The simulator is used to update the visualisation
simulator = Simulator(
    diagram_task_space, 
    diagram_context_task_space
)
simulator.Initialize()


# SceneGraph inspectors for highlighting geometry pairs.
# This means that we can highlight the collision pairs.
# The query object is used to get the collision pairs, and the model_inspector is used to highlight them.
model_inspector = scene_graph.model_inspector()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(diagram_context_task_space))


q_star = np.zeros(plant.num_positions()-1)

# Construct the Rational Forward Kinematics
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star
)

rat_forward_kin = RationalForwardKinematics(plant)
s_variables = sym.Variables(rat_forward_kin.s())
print(rat_forward_kin.s())

INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003


3
['bottle_cap_cap_to_base_q', 'Schunk_Gripper_base_revolute_joint_q', 'Schunk_Gripper_left_finger_sliding_joint_x']
[Variable('s[0]', Continuous) Variable('s[1]', Continuous)
 Variable('s[2]', Continuous)]


## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the three degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [7]:
sliders = []

for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    step = (q_high - q_low) / 100
    sliders.append(widgets.FloatSlider(
        min=q_low, max=q_high, 
        value=0, step=step, 
        description=f"q{i}"))
    
q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    print(visualizer.check_collision_q_by_ik(q))
    plant.SetPositions(plant_context, q)
    diagram_task_space.ForcedPublish(diagram_context_task_space)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=3.14, min=-3.14, step=0.06280000000000001)

FloatSlider(value=0.0, description='q1', max=1.0, min=-1.0, step=0.02)

FloatSlider(value=0.0, description='q2', max=0.0, min=-0.055, step=0.00055)

NameError: name 'visualizer' is not defined

## Set up the solver for certifying the region. 

It is **very** important that you upload a MOSEK license into the file `mosek.lic` in order to run the rest of the notebook. If you do not, the solver `SCS` will be used which is much slower and likely will be too inaccurate to solve many of the problems.

In [7]:
import warnings
from pydrake.common.deprecation import DrakeDeprecationWarning

# os.environ["MOSEKLM_LICENSE_FILE"] = "mosek.lic"
with open("mosec.lic", 'r') as f:
    contents = f.read()
    mosek_file_not_empty = contents != ''
print(mosek_file_not_empty)

solver_id = MosekSolver.id() if MosekSolver().available() and mosek_file_not_empty else ScsSolver.id()


# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.parallelism = Parallelism()
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = False
find_separation_certificate_given_polytope_options.solver_id = solver_id

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-4
find_polytope_given_lagrangian_option.solver_id = solver_id


bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 10
bilinear_alternation_options.convergence_tol = 1e-3
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.scale_min = 1e-3
binary_search_options.scale_max = 1.0
binary_search_options.max_iter = 5

True


## Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions using Algorithm 2 which we will later certify.

In [8]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

gripper = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

plant.Finalize()

In [9]:
B_cap = plant.GetBodyByName("cap_link", cap)
B_gripper = plant.GetBodyByName("body", gripper)


In [10]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

NameError: name 'meshcat' is not defined

In [ ]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

p_GgraspO = [0, .115, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2).multiply(RotationMatrix.MakeZRotation(np.pi / 2))

X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

plant.SetFreeBodyPose(plant_context, B_gripper, X_WGgrasp)

plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.033
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.033
)

In [ ]:
diagram.ForcedPublish(context)